In this notebook I train the pre-trained detection models EfficientDet-Lite (info below) to detect climbing holds (currently only one class).  
Then model is evaluated, exported as a tensorflow lite model and evaluated again (as a lite model).  

Need to run `00_prepare_data_for_training_and_view.ipynb` first.  

run this notebook on env:   
conda activate tf2_gpu_berta  

Env main installations:
```
conda install pip  
sudo apt -y install libportaudio2  
~/miniconda3/envs/tf2_gpu_berta/bin/pip install tflite-model-maker  
~/miniconda3/envs/tf2_gpu_berta/bin/pip install -q pycocotools  
~/miniconda3/envs/tf2_gpu_berta/bin/pip install -q opencv-python-headless==4.5.1.48
~/miniconda3/envs/tf2_gpu_berta/bin/pip uninstall -y tensorflow && ~/miniconda3/envs/tf2_gpu_berta/bin/pip install -q tensorflow==2.8.0
```

Notebook source: https://www.tensorflow.org/lite/models/modify/model_maker/object_detection

In [1]:
##### Copyright 2021 The TensorFlow Authors.

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
import numpy as np

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [3]:
import os
import pandas as pd
import shutil

In [4]:
# Authenticate google cloud:
from google.cloud.bigquery.client import Client

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'auto_key.json'
bq_client = Client()

## Load the data

In [5]:
csv_name = 'training_climbing_hall_sameLabelAllHolds.csv'

Model Maker takes input data in a CSV format   
(see 00_prepare_data_for_training_and_view.ipynb for more details)  

You can load the CSV file directly from Google Cloud Storage, but you don't need to keep your images on Google Cloud to use Model Maker. You can specify a local CSV file on your computer, and Model Maker will work just fine.


In [6]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(
    f'gs://training_csvs/{csv_name}')

In [7]:
type(train_data)

tensorflow_examples.lite.model_maker.core.data_util.object_detector_dataloader.DataLoader

## Choosing a pretrained model:

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture. (details in the paper)

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Pixel 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


### Customize the EfficientDet model hyperparameters

The model and training pipline parameters you can adjust are:

* `model_dir`: The location to save the model checkpoint files. If not set, a temporary directory will be used.
* `steps_per_execution`: Number of steps per training execution.
* `moving_average_decay`: Float. The decay to use for maintaining moving averages of the trained parameters.
* `var_freeze_expr`: The regular expression to map the prefix name of variables to be frozen which means remaining the same during training. More specific, use `re.match(var_freeze_expr, variable_name)` in the codebase to map the variables to be frozen.
* `tflite_max_detections`: integer, 25 by default. The max number of output detections in the TFLite model.
* `strategy`:  A string specifying which distribution strategy to use. Accepted values are 'tpu', 'gpus', None. tpu' means to use TPUStrategy. 'gpus' mean to use MirroredStrategy for multi-gpus. If None, use TF default with OneDeviceStrategy.
* `tpu`:  The Cloud TPU to use for training. This should be either the name used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 url.
* `use_xla`: Use XLA even if strategy is not tpu. If strategy is tpu, always use XLA, and this flag has no effect.
* `profile`: Enable profile mode.
* `debug`: Enable debug mode.

Other parameters that can be adjusted is shown in [hparams_config.py](https://github.com/google/automl/blob/df451765d467c5ed78bbdfd632810bc1014b123e/efficientdet/hparams_config.py#L170).


For instance, you can set the `var_freeze_expr='efficientnet'` which freezes the variables with name prefix `efficientnet` (default is `'(efficientnet|fpn_cells|resample_p6)'`). This allows the model to freeze untrainable variables and keep their value the same through training.

```python
spec = model_spec.get('efficientdet_lite0')
spec.config.var_freeze_expr = 'efficientnet'
```

### Tune the training hyperparameters

The `create` function is the driver function that the Model Maker library uses to create models. The `model_spec` parameter defines the model specification. The `object_detector.EfficientDetSpec` class is currently supported. The `create` function comprises of the following steps:

1. Creates the model for the object detection according to `model_spec`.
2. Trains the model.  The default epochs and the default batch size are set by the `epochs` and `batch_size` variables in the `model_spec` object.
You can also tune the training hyperparameters like `epochs` and `batch_size` that affect the model accuracy. For instance,

*   `epochs`: Integer, 50 by default. More epochs could achieve better accuracy, but may lead to overfitting.
*   `batch_size`: Integer, 64 by default. The number of samples to use in one training step.
*   `train_whole_model`: Boolean, False by default. If true, train the whole model. Otherwise, only train the layers that do not match `var_freeze_expr`.

For example, you can train with less epochs and only the head layer. You can increase the number of epochs for better results.

```python
model = object_detector.create(train_data, model_spec=spec, epochs=10, validation_data=validation_data)
```

## EfficientDet-Lite0

In [8]:
model_num = 0
model_name = f'efficientdet_lite{model_num}'

epochs = 90

In [9]:
## First train the smallest model
spec = model_spec.get(model_name)

spec.config.model_dir = model_name

2022-08-04 15:57:57.897519: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-04 15:57:58.712205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23133 MB memory:  -> device: 0, name: Quadro P6000, pci bus id: 0000:03:00.0, compute capability: 6.1


### Training
EfficientDet-Lite0 model uses epochs = 50 by default.  
need to look at the validation accuracy during training and stop early to avoid overfitting. (no automatic early stopping?)  
Set train_whole_model=True to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [10]:
model = object_detector.create(train_data, 
                               model_spec=spec, 
                               batch_size=8,
                               epochs=epochs,
                               train_whole_model=True, 
                               validation_data=validation_data)

Epoch 1/90


2022-08-04 15:58:37.954303: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-08-04 15:58:38.236061: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-04 15:58:38.237178: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-04 15:58:38.237217: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-08-04 15:58:38.238549: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-04 15:58:38.238691: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


15/15 [==============================] - 36s 496ms/step - det_loss: 2.0355 - cls_loss: 1.0990 - box_loss: 0.0187 - reg_l2_loss: 0.0630 - loss: 2.0985 - learning_rate: 0.0090 - gradient_norm: 0.8619 - val_det_loss: 1.7008 - val_cls_loss: 1.0396 - val_box_loss: 0.0132 - val_reg_l2_loss: 0.0630 - val_loss: 1.7638
Epoch 2/90
15/15 [==============================] - 4s 287ms/step - det_loss: 1.6685 - cls_loss: 0.8904 - box_loss: 0.0156 - reg_l2_loss: 0.0630 - loss: 1.7315 - learning_rate: 0.0100 - gradient_norm: 1.2049 - val_det_loss: 1.1647 - val_cls_loss: 0.5995 - val_box_loss: 0.0113 - val_reg_l2_loss: 0.0630 - val_loss: 1.2277
Epoch 3/90
15/15 [==============================] - 4s 292ms/step - det_loss: 1.1764 - cls_loss: 0.4837 - box_loss: 0.0139 - reg_l2_loss: 0.0630 - loss: 1.2394 - learning_rate: 0.0100 - gradient_norm: 1.8119 - val_det_loss: 0.9381 - val_cls_loss: 0.4596 - val_box_loss: 0.0096 - val_reg_l2_loss: 0.0630 - val_loss: 1.0011
Epoch 4/90
15/15 [==========================

15/15 [==============================] - 4s 291ms/step - det_loss: 0.4001 - cls_loss: 0.2096 - box_loss: 0.0038 - reg_l2_loss: 0.0631 - loss: 0.4632 - learning_rate: 0.0081 - gradient_norm: 1.3237 - val_det_loss: 0.3476 - val_cls_loss: 0.1973 - val_box_loss: 0.0030 - val_reg_l2_loss: 0.0631 - val_loss: 0.4107
Epoch 27/90
15/15 [==============================] - 5s 345ms/step - det_loss: 0.3880 - cls_loss: 0.2044 - box_loss: 0.0037 - reg_l2_loss: 0.0631 - loss: 0.4511 - learning_rate: 0.0080 - gradient_norm: 1.3362 - val_det_loss: 0.3585 - val_cls_loss: 0.2125 - val_box_loss: 0.0029 - val_reg_l2_loss: 0.0631 - val_loss: 0.4216
Epoch 28/90
15/15 [==============================] - 4s 295ms/step - det_loss: 0.3798 - cls_loss: 0.2017 - box_loss: 0.0036 - reg_l2_loss: 0.0631 - loss: 0.4429 - learning_rate: 0.0078 - gradient_norm: 1.2600 - val_det_loss: 0.3436 - val_cls_loss: 0.2080 - val_box_loss: 0.0027 - val_reg_l2_loss: 0.0631 - val_loss: 0.4067
Epoch 29/90
15/15 [========================

15/15 [==============================] - 5s 354ms/step - det_loss: 0.3446 - cls_loss: 0.1767 - box_loss: 0.0034 - reg_l2_loss: 0.0630 - loss: 0.4076 - learning_rate: 0.0039 - gradient_norm: 1.2159 - val_det_loss: 0.2897 - val_cls_loss: 0.1757 - val_box_loss: 0.0023 - val_reg_l2_loss: 0.0630 - val_loss: 0.3527
Epoch 52/90
15/15 [==============================] - 4s 295ms/step - det_loss: 0.3696 - cls_loss: 0.1880 - box_loss: 0.0036 - reg_l2_loss: 0.0630 - loss: 0.4326 - learning_rate: 0.0038 - gradient_norm: 1.8900 - val_det_loss: 0.2896 - val_cls_loss: 0.1771 - val_box_loss: 0.0022 - val_reg_l2_loss: 0.0630 - val_loss: 0.3526
Epoch 53/90
15/15 [==============================] - 4s 288ms/step - det_loss: 0.3548 - cls_loss: 0.1828 - box_loss: 0.0034 - reg_l2_loss: 0.0630 - loss: 0.4178 - learning_rate: 0.0036 - gradient_norm: 1.2767 - val_det_loss: 0.2872 - val_cls_loss: 0.1762 - val_box_loss: 0.0022 - val_reg_l2_loss: 0.0630 - val_loss: 0.3502
Epoch 54/90
15/15 [========================

15/15 [==============================] - 4s 302ms/step - det_loss: 0.3145 - cls_loss: 0.1638 - box_loss: 0.0030 - reg_l2_loss: 0.0630 - loss: 0.3775 - learning_rate: 5.5764e-04 - gradient_norm: 1.1177 - val_det_loss: 0.2781 - val_cls_loss: 0.1712 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0630 - val_loss: 0.3411
Epoch 77/90
15/15 [==============================] - 4s 296ms/step - det_loss: 0.3287 - cls_loss: 0.1744 - box_loss: 0.0031 - reg_l2_loss: 0.0630 - loss: 0.3917 - learning_rate: 4.7944e-04 - gradient_norm: 1.1724 - val_det_loss: 0.2816 - val_cls_loss: 0.1727 - val_box_loss: 0.0022 - val_reg_l2_loss: 0.0630 - val_loss: 0.3446
Epoch 78/90
15/15 [==============================] - 5s 347ms/step - det_loss: 0.3252 - cls_loss: 0.1699 - box_loss: 0.0031 - reg_l2_loss: 0.0630 - loss: 0.3882 - learning_rate: 4.0688e-04 - gradient_norm: 1.1144 - val_det_loss: 0.2791 - val_cls_loss: 0.1724 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0630 - val_loss: 0.3421
Epoch 79/90
15/15 [============

In [11]:
## Need to update Nvidia-Driver version. Schade

In [12]:
## Looks like maybe the learning rate should be lower for epoch ~60, need to adjust 
## (link to all hyperparameters above)

## If as is, should have probably stopped at epoch ~80.

### Evaluating

From https://cocodataset.org/#detection-eval:  

Average Precision (AP):  
AP  
% AP at IoU=.50:.05:.95 (primary challenge metric)  
APIoU=.50  
% AP at IoU=.50 (PASCAL VOC metric)  
APIoU=.75  
% AP at IoU=.75 (strict metric)  
AP Across Scales:  
APsmall  
% AP for small objects: area < 322  
APmedium  
% AP for medium objects: 322 < area < 962  
APlarge  
% AP for large objects: area > 962  
Average Recall (AR):  
ARmax=1  
% AR given 1 detection per image  
ARmax=10  
% AR given 10 detections per image  
ARmax=100  
% AR given 100 detections per image  
AR Across Scales:  
ARsmall  
% AR for small objects: area < 322  
ARmedium  
% AR for medium objects: 322 < area < 962  
ARlarge  
% AR for large objects: area > 962  


In [13]:
%%capture cap --no-stderr

model.evaluate(test_data)

In [15]:
with open(f'evaluation_EfficientDet{model_num}_preLite.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

1/1 [==============================] - 4s 4s/step



{'AP': 0.36688063,
 'AP50': 0.6299745,
 'AP75': 0.38016248,
 'APs': 0.2575144,
 'APm': 0.58892274,
 'APl': 0.6410678,
 'ARmax1': 0.020284697,
 'ARmax10': 0.17224199,
 'ARmax100': 0.45017794,
 'ARs': 0.33687,
 'ARm': 0.67134506,
 'ARl': 0.76666665,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.36688063}

### Export as a TensorFlow Lite model

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is full integer quantization.

In [16]:
model.export(export_dir='.', tflite_filename=f'model{model_num}.tflite')

2022-08-04 16:05:58.434956: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-08-04 16:06:29.341405: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-08-04 16:06:36.764834: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-08-04 16:06:36.764877: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-08-04 16:06:36.766052: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpnh5aut_6
2022-08-04 16:06:36.888233: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-08-04 16:06:36.888281: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if pre

In [17]:
## Those warnings are the default output - info below

### Export to different formats

The export formats can be one or a list of the following:

*   `ExportFormat.TFLITE`
*   `ExportFormat.LABEL`
*   `ExportFormat.SAVED_MODEL`




By default, it exports only the TensorFlow Lite model file containing the model [metadata](https://www.tensorflow.org/lite/models/convert/metadata) so that you can later use in an on-device ML application. The label file is embedded in metadata.

In many on-device ML application, the model size is an important factor. Therefore, it is recommended that you quantize the model to make it smaller and potentially run faster. As for EfficientDet-Lite models, full integer quantization  is used to quantize the model by default. Please refer to [Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) for more detail.

```python
model.export(export_dir='.')
```

You can also choose to export other files related to the model for better examination. For instance, exporting both the saved model and the label file as follows:
```python
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])
```

### Customize Post-training quantization on the TensorFlow Lite model

[Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) is a conversion technique that can reduce model size and inference latency, while also improving CPU and hardware accelerator inference speed, with a little degradation in model accuracy. Thus, it's widely used to optimize the model.

Model Maker library applies a default post-training quantization techique when exporting the model. If you want to customize post-training quantization, Model Maker supports multiple post-training quantization options using [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) as well. Let's take float16 quantization as an instance. First, define the quantization config.

```python
config = QuantizationConfig.for_float16()
```


Then we export the TensorFlow Lite model with such configuration.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Read more

You can read our [object detection](https://www.tensorflow.org/lite/examples/object_detection/overview) example to learn technical details. For more information, please refer to:

*   TensorFlow Lite Model Maker [guide](https://www.tensorflow.org/lite/models/modify/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).
*   Task Library: [ObjectDetector](https://www.tensorflow.org/lite/inference_with_metadata/task_library/object_detector) for deployment.
*   The end-to-end reference apps: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/android), [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/ios), and [Raspberry PI](https://github.com/tensorflow/examples/tree/master/lite/examples/object_detection/raspberry_pi).



### Evaluate the TensorFlow Lite model

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [18]:
%%capture cap --no-stderr

model.evaluate_tflite(f'model{model_num}.tflite', test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [19]:
## Thats much much worse. need to figure it out / another model.

In [20]:
with open(f'evaluation_EfficientDet-Lite{model_num}.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

15/15 [==============================] - 35s 2s/step



{'AP': 0.28082144,
 'AP50': 0.447756,
 'AP75': 0.31311336,
 'APs': 0.15442218,
 'APm': 0.54207397,
 'APl': 0.5846207,
 'ARmax1': 0.019928826,
 'ARmax10': 0.17081851,
 'ARmax100': 0.3144128,
 'ARs': 0.17453581,
 'ARm': 0.5888889,
 'ARl': 0.70666665,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.28082144}

## EfficientDet-Lite1

In [21]:
model_num = 1
model_name = f'efficientdet_lite{model_num}'

epochs = 90

## First train the smallest model
spec = model_spec.get(model_name)
spec.config.model_dir = model_name

model = object_detector.create(train_data, 
                               model_spec=spec, 
                               batch_size=8,
                               epochs=epochs,
                               train_whole_model=True, 
                               validation_data=validation_data)

Epoch 1/90
15/15 [==============================] - 44s 595ms/step - det_loss: 1.8958 - cls_loss: 1.0795 - box_loss: 0.0163 - reg_l2_loss: 0.0706 - loss: 1.9664 - learning_rate: 0.0090 - gradient_norm: 0.8498 - val_det_loss: 1.6072 - val_cls_loss: 1.0084 - val_box_loss: 0.0120 - val_reg_l2_loss: 0.0706 - val_loss: 1.6778
Epoch 2/90
15/15 [==============================] - 6s 410ms/step - det_loss: 1.4934 - cls_loss: 0.7873 - box_loss: 0.0141 - reg_l2_loss: 0.0706 - loss: 1.5640 - learning_rate: 0.0100 - gradient_norm: 1.5173 - val_det_loss: 1.1186 - val_cls_loss: 0.6334 - val_box_loss: 0.0097 - val_reg_l2_loss: 0.0706 - val_loss: 1.1892
Epoch 3/90
15/15 [==============================] - 6s 402ms/step - det_loss: 1.0089 - cls_loss: 0.4291 - box_loss: 0.0116 - reg_l2_loss: 0.0706 - loss: 1.0795 - learning_rate: 0.0100 - gradient_norm: 1.6905 - val_det_loss: 0.7332 - val_cls_loss: 0.3496 - val_box_loss: 0.0077 - val_reg_l2_loss: 0.0706 - val_loss: 0.8038
Epoch 4/90
15/15 [===============

15/15 [==============================] - 6s 398ms/step - det_loss: 0.3583 - cls_loss: 0.1924 - box_loss: 0.0033 - reg_l2_loss: 0.0706 - loss: 0.4289 - learning_rate: 0.0081 - gradient_norm: 1.3566 - val_det_loss: 0.2765 - val_cls_loss: 0.1718 - val_box_loss: 0.0021 - val_reg_l2_loss: 0.0706 - val_loss: 0.3472
Epoch 27/90
15/15 [==============================] - 6s 403ms/step - det_loss: 0.3423 - cls_loss: 0.1926 - box_loss: 0.0030 - reg_l2_loss: 0.0706 - loss: 0.4129 - learning_rate: 0.0080 - gradient_norm: 1.4467 - val_det_loss: 0.2626 - val_cls_loss: 0.1657 - val_box_loss: 0.0019 - val_reg_l2_loss: 0.0706 - val_loss: 0.3332
Epoch 28/90
15/15 [==============================] - 6s 398ms/step - det_loss: 0.3279 - cls_loss: 0.1850 - box_loss: 0.0029 - reg_l2_loss: 0.0706 - loss: 0.3985 - learning_rate: 0.0078 - gradient_norm: 1.3194 - val_det_loss: 0.2675 - val_cls_loss: 0.1651 - val_box_loss: 0.0020 - val_reg_l2_loss: 0.0706 - val_loss: 0.3381
Epoch 29/90
15/15 [========================

15/15 [==============================] - 6s 396ms/step - det_loss: 0.2801 - cls_loss: 0.1618 - box_loss: 0.0024 - reg_l2_loss: 0.0706 - loss: 0.3506 - learning_rate: 0.0039 - gradient_norm: 1.1240 - val_det_loss: 0.2239 - val_cls_loss: 0.1439 - val_box_loss: 0.0016 - val_reg_l2_loss: 0.0706 - val_loss: 0.2945
Epoch 52/90
15/15 [==============================] - 6s 408ms/step - det_loss: 0.3002 - cls_loss: 0.1654 - box_loss: 0.0027 - reg_l2_loss: 0.0706 - loss: 0.3708 - learning_rate: 0.0038 - gradient_norm: 1.4439 - val_det_loss: 0.2231 - val_cls_loss: 0.1429 - val_box_loss: 0.0016 - val_reg_l2_loss: 0.0706 - val_loss: 0.2937
Epoch 53/90
15/15 [==============================] - 6s 393ms/step - det_loss: 0.2821 - cls_loss: 0.1626 - box_loss: 0.0024 - reg_l2_loss: 0.0706 - loss: 0.3527 - learning_rate: 0.0036 - gradient_norm: 1.1952 - val_det_loss: 0.2318 - val_cls_loss: 0.1484 - val_box_loss: 0.0017 - val_reg_l2_loss: 0.0706 - val_loss: 0.3024
Epoch 54/90
15/15 [========================

15/15 [==============================] - 7s 478ms/step - det_loss: 0.2690 - cls_loss: 0.1518 - box_loss: 0.0023 - reg_l2_loss: 0.0705 - loss: 0.3396 - learning_rate: 5.5764e-04 - gradient_norm: 0.9706 - val_det_loss: 0.2170 - val_cls_loss: 0.1414 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0705 - val_loss: 0.2876
Epoch 77/90
15/15 [==============================] - 6s 402ms/step - det_loss: 0.2879 - cls_loss: 0.1641 - box_loss: 0.0025 - reg_l2_loss: 0.0705 - loss: 0.3584 - learning_rate: 4.7944e-04 - gradient_norm: 1.2401 - val_det_loss: 0.2163 - val_cls_loss: 0.1410 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0705 - val_loss: 0.2869
Epoch 78/90
15/15 [==============================] - 6s 388ms/step - det_loss: 0.2769 - cls_loss: 0.1595 - box_loss: 0.0023 - reg_l2_loss: 0.0705 - loss: 0.3475 - learning_rate: 4.0688e-04 - gradient_norm: 1.0892 - val_det_loss: 0.2169 - val_cls_loss: 0.1408 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0705 - val_loss: 0.2874
Epoch 79/90
15/15 [============

In [22]:
%%capture cap --no-stderr
model.evaluate(test_data)

In [ ]:
with open(f'evaluation_EfficientDet{model_num}_preLite.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

In [23]:
model.export(export_dir='.', tflite_filename=f'model{model_num}.tflite')

2022-08-04 16:18:55.304247: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-08-04 16:19:11.394726: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-08-04 16:19:11.394776: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-08-04 16:19:11.395050: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpg228jg7z
2022-08-04 16:19:11.543156: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-08-04 16:19:11.543227: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpg228jg7z
2022-08-04 16:19:11.992560: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-08-04 16:19:14.636208: I tensorflow/cc/saved_

In [24]:
%%capture cap --no-stderr
model.evaluate_tflite(f'model{model_num}.tflite', test_data)

In [25]:
with open(f'evaluation_EfficientDet-Lite{model_num}.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

15/15 [==============================] - 70s 5s/step



{'AP': 0.33648214,
 'AP50': 0.4961116,
 'AP75': 0.3785436,
 'APs': 0.19117028,
 'APm': 0.633111,
 'APl': 0.5997047,
 'ARmax1': 0.020818505,
 'ARmax10': 0.19430605,
 'ARmax100': 0.3613879,
 'ARs': 0.20371352,
 'ARm': 0.6766082,
 'ARl': 0.70666665,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.33648214}

## EfficientDet-Lite2

In [26]:
model_num = 2
model_name = f'efficientdet_lite{model_num}'

epochs = 90

## First train the smallest model
spec = model_spec.get(model_name)
spec.config.model_dir = model_name

model = object_detector.create(train_data, 
                               model_spec=spec, 
                               batch_size=8,
                               epochs=epochs,
                               train_whole_model=True, 
                               validation_data=validation_data)

Epoch 1/90
15/15 [==============================] - 49s 749ms/step - det_loss: 1.8289 - cls_loss: 1.0839 - box_loss: 0.0149 - reg_l2_loss: 0.0759 - loss: 1.9048 - learning_rate: 0.0090 - gradient_norm: 0.7804 - val_det_loss: 1.5673 - val_cls_loss: 1.0098 - val_box_loss: 0.0111 - val_reg_l2_loss: 0.0759 - val_loss: 1.6431
Epoch 2/90
15/15 [==============================] - 8s 537ms/step - det_loss: 1.4281 - cls_loss: 0.7732 - box_loss: 0.0131 - reg_l2_loss: 0.0759 - loss: 1.5040 - learning_rate: 0.0100 - gradient_norm: 1.6230 - val_det_loss: 1.5537 - val_cls_loss: 1.0858 - val_box_loss: 0.0094 - val_reg_l2_loss: 0.0759 - val_loss: 1.6297
Epoch 3/90
15/15 [==============================] - 8s 552ms/step - det_loss: 0.9436 - cls_loss: 0.4299 - box_loss: 0.0103 - reg_l2_loss: 0.0759 - loss: 1.0195 - learning_rate: 0.0100 - gradient_norm: 1.7746 - val_det_loss: 0.8124 - val_cls_loss: 0.4458 - val_box_loss: 0.0073 - val_reg_l2_loss: 0.0759 - val_loss: 0.8883
Epoch 4/90
15/15 [===============

15/15 [==============================] - 9s 579ms/step - det_loss: 0.3065 - cls_loss: 0.1771 - box_loss: 0.0026 - reg_l2_loss: 0.0759 - loss: 0.3825 - learning_rate: 0.0081 - gradient_norm: 1.3738 - val_det_loss: 0.2522 - val_cls_loss: 0.1558 - val_box_loss: 0.0019 - val_reg_l2_loss: 0.0759 - val_loss: 0.3282
Epoch 27/90
15/15 [==============================] - 8s 545ms/step - det_loss: 0.3376 - cls_loss: 0.1831 - box_loss: 0.0031 - reg_l2_loss: 0.0759 - loss: 0.4135 - learning_rate: 0.0080 - gradient_norm: 1.6716 - val_det_loss: 0.2531 - val_cls_loss: 0.1597 - val_box_loss: 0.0019 - val_reg_l2_loss: 0.0759 - val_loss: 0.3290
Epoch 28/90
15/15 [==============================] - 8s 541ms/step - det_loss: 0.3383 - cls_loss: 0.1855 - box_loss: 0.0031 - reg_l2_loss: 0.0759 - loss: 0.4142 - learning_rate: 0.0078 - gradient_norm: 1.6323 - val_det_loss: 0.2563 - val_cls_loss: 0.1587 - val_box_loss: 0.0020 - val_reg_l2_loss: 0.0759 - val_loss: 0.3322
Epoch 29/90
15/15 [========================

15/15 [==============================] - 9s 646ms/step - det_loss: 0.3075 - cls_loss: 0.1649 - box_loss: 0.0029 - reg_l2_loss: 0.0759 - loss: 0.3833 - learning_rate: 0.0039 - gradient_norm: 1.4144 - val_det_loss: 0.2173 - val_cls_loss: 0.1392 - val_box_loss: 0.0016 - val_reg_l2_loss: 0.0759 - val_loss: 0.2932
Epoch 52/90
15/15 [==============================] - 8s 557ms/step - det_loss: 0.2581 - cls_loss: 0.1514 - box_loss: 0.0021 - reg_l2_loss: 0.0759 - loss: 0.3340 - learning_rate: 0.0038 - gradient_norm: 1.0170 - val_det_loss: 0.2146 - val_cls_loss: 0.1374 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0759 - val_loss: 0.2905
Epoch 53/90
15/15 [==============================] - 8s 569ms/step - det_loss: 0.2540 - cls_loss: 0.1471 - box_loss: 0.0021 - reg_l2_loss: 0.0759 - loss: 0.3299 - learning_rate: 0.0036 - gradient_norm: 0.9863 - val_det_loss: 0.2116 - val_cls_loss: 0.1365 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0759 - val_loss: 0.2875
Epoch 54/90
15/15 [========================

15/15 [==============================] - 8s 580ms/step - det_loss: 0.2452 - cls_loss: 0.1479 - box_loss: 0.0019 - reg_l2_loss: 0.0758 - loss: 0.3210 - learning_rate: 5.5764e-04 - gradient_norm: 1.0582 - val_det_loss: 0.2071 - val_cls_loss: 0.1333 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0758 - val_loss: 0.2830
Epoch 77/90
15/15 [==============================] - 10s 661ms/step - det_loss: 0.2788 - cls_loss: 0.1526 - box_loss: 0.0025 - reg_l2_loss: 0.0758 - loss: 0.3546 - learning_rate: 4.7944e-04 - gradient_norm: 1.2798 - val_det_loss: 0.2077 - val_cls_loss: 0.1335 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0758 - val_loss: 0.2835
Epoch 78/90
15/15 [==============================] - 9s 581ms/step - det_loss: 0.2689 - cls_loss: 0.1533 - box_loss: 0.0023 - reg_l2_loss: 0.0758 - loss: 0.3447 - learning_rate: 4.0688e-04 - gradient_norm: 1.1539 - val_det_loss: 0.2071 - val_cls_loss: 0.1334 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0758 - val_loss: 0.2829
Epoch 79/90
15/15 [===========

In [27]:
%%capture cap --no-stderr
model.evaluate(test_data)

In [29]:
with open(f'evaluation_EfficientDet{model_num}_preLite.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

1/1 [==============================] - 5s 5s/step



{'AP': 0.52801114,
 'AP50': 0.8342187,
 'AP75': 0.5720881,
 'APs': 0.44196653,
 'APm': 0.7127014,
 'APl': 0.6575063,
 'ARmax1': 0.02241993,
 'ARmax10': 0.20587188,
 'ARmax100': 0.6005338,
 'ARs': 0.5127321,
 'ARm': 0.7754386,
 'ARl': 0.81333333,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.52801114}

In [30]:
model.export(export_dir='.', tflite_filename=f'model{model_num}.tflite')

2022-08-04 16:36:50.263969: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-08-04 16:37:01.165101: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-08-04 16:37:01.165150: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-08-04 16:37:01.165420: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpwg_t353m
2022-08-04 16:37:01.328245: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-08-04 16:37:01.328301: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpwg_t353m
2022-08-04 16:37:01.891082: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-08-04 16:37:04.770495: I tensorflow/cc/saved_

In [31]:
%%capture cap --no-stderr
model.evaluate_tflite(f'model{model_num}.tflite', test_data)

In [32]:
with open(f'evaluation_EfficientDet-Lite{model_num}.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

15/15 [==============================] - 118s 8s/step



{'AP': 0.37148884,
 'AP50': 0.52957815,
 'AP75': 0.43367568,
 'APs': 0.23589589,
 'APm': 0.6532577,
 'APl': 0.6276683,
 'ARmax1': 0.021886121,
 'ARmax10': 0.2024911,
 'ARmax100': 0.39733097,
 'ARs': 0.24827586,
 'ARm': 0.69005847,
 'ARl': 0.78,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.37148884}

## EfficientDet-Lite3

In [33]:
model_num = 3
model_name = f'efficientdet_lite{model_num}'

epochs = 90

## First train the smallest model
spec = model_spec.get(model_name)
spec.config.model_dir = model_name

model = object_detector.create(train_data, 
                               model_spec=spec, 
                               batch_size=8,
                               epochs=epochs,
                               train_whole_model=True, 
                               validation_data=validation_data)

Epoch 1/90
15/15 [==============================] - 60s 966ms/step - det_loss: 1.8136 - cls_loss: 1.1088 - box_loss: 0.0141 - reg_l2_loss: 0.0926 - loss: 1.9061 - learning_rate: 0.0090 - gradient_norm: 0.6642 - val_det_loss: 1.6078 - val_cls_loss: 1.0538 - val_box_loss: 0.0111 - val_reg_l2_loss: 0.0925 - val_loss: 1.7003
Epoch 2/90
15/15 [==============================] - 11s 756ms/step - det_loss: 1.4754 - cls_loss: 0.8622 - box_loss: 0.0123 - reg_l2_loss: 0.0926 - loss: 1.5680 - learning_rate: 0.0100 - gradient_norm: 1.3970 - val_det_loss: 1.0998 - val_cls_loss: 0.6939 - val_box_loss: 0.0081 - val_reg_l2_loss: 0.0926 - val_loss: 1.1923
Epoch 3/90
15/15 [==============================] - 11s 734ms/step - det_loss: 0.8902 - cls_loss: 0.4289 - box_loss: 0.0092 - reg_l2_loss: 0.0926 - loss: 0.9828 - learning_rate: 0.0100 - gradient_norm: 1.9639 - val_det_loss: 0.6732 - val_cls_loss: 0.4020 - val_box_loss: 0.0054 - val_reg_l2_loss: 0.0926 - val_loss: 0.7658
Epoch 4/90
15/15 [=============

15/15 [==============================] - 13s 880ms/step - det_loss: 0.2705 - cls_loss: 0.1595 - box_loss: 0.0022 - reg_l2_loss: 0.0925 - loss: 0.3630 - learning_rate: 0.0081 - gradient_norm: 1.2903 - val_det_loss: 0.1936 - val_cls_loss: 0.1315 - val_box_loss: 0.0012 - val_reg_l2_loss: 0.0925 - val_loss: 0.2862
Epoch 27/90
15/15 [==============================] - 11s 759ms/step - det_loss: 0.2533 - cls_loss: 0.1574 - box_loss: 0.0019 - reg_l2_loss: 0.0925 - loss: 0.3459 - learning_rate: 0.0080 - gradient_norm: 1.3166 - val_det_loss: 0.1856 - val_cls_loss: 0.1268 - val_box_loss: 0.0012 - val_reg_l2_loss: 0.0925 - val_loss: 0.2782
Epoch 28/90
15/15 [==============================] - 11s 763ms/step - det_loss: 0.2449 - cls_loss: 0.1538 - box_loss: 0.0018 - reg_l2_loss: 0.0925 - loss: 0.3375 - learning_rate: 0.0078 - gradient_norm: 1.2391 - val_det_loss: 0.1937 - val_cls_loss: 0.1346 - val_box_loss: 0.0012 - val_reg_l2_loss: 0.0925 - val_loss: 0.2862
Epoch 29/90
15/15 [=====================

15/15 [==============================] - 11s 738ms/step - det_loss: 0.2063 - cls_loss: 0.1307 - box_loss: 0.0015 - reg_l2_loss: 0.0924 - loss: 0.2988 - learning_rate: 0.0039 - gradient_norm: 1.0724 - val_det_loss: 0.1701 - val_cls_loss: 0.1191 - val_box_loss: 0.0010 - val_reg_l2_loss: 0.0924 - val_loss: 0.2626
Epoch 52/90
15/15 [==============================] - 11s 727ms/step - det_loss: 0.2115 - cls_loss: 0.1330 - box_loss: 0.0016 - reg_l2_loss: 0.0924 - loss: 0.3039 - learning_rate: 0.0038 - gradient_norm: 1.0877 - val_det_loss: 0.1684 - val_cls_loss: 0.1163 - val_box_loss: 0.0010 - val_reg_l2_loss: 0.0924 - val_loss: 0.2608
Epoch 53/90
15/15 [==============================] - 11s 739ms/step - det_loss: 0.2056 - cls_loss: 0.1283 - box_loss: 0.0015 - reg_l2_loss: 0.0924 - loss: 0.2980 - learning_rate: 0.0036 - gradient_norm: 1.0122 - val_det_loss: 0.1701 - val_cls_loss: 0.1172 - val_box_loss: 0.0011 - val_reg_l2_loss: 0.0924 - val_loss: 0.2625
Epoch 54/90
15/15 [=====================

15/15 [==============================] - 11s 739ms/step - det_loss: 0.2046 - cls_loss: 0.1271 - box_loss: 0.0015 - reg_l2_loss: 0.0924 - loss: 0.2970 - learning_rate: 5.5764e-04 - gradient_norm: 1.1862 - val_det_loss: 0.1611 - val_cls_loss: 0.1112 - val_box_loss: 9.9757e-04 - val_reg_l2_loss: 0.0924 - val_loss: 0.2535
Epoch 77/90
15/15 [==============================] - 11s 753ms/step - det_loss: 0.1936 - cls_loss: 0.1227 - box_loss: 0.0014 - reg_l2_loss: 0.0924 - loss: 0.2860 - learning_rate: 4.7944e-04 - gradient_norm: 0.9088 - val_det_loss: 0.1604 - val_cls_loss: 0.1114 - val_box_loss: 9.8173e-04 - val_reg_l2_loss: 0.0924 - val_loss: 0.2528
Epoch 78/90
15/15 [==============================] - 11s 756ms/step - det_loss: 0.1905 - cls_loss: 0.1207 - box_loss: 0.0014 - reg_l2_loss: 0.0924 - loss: 0.2829 - learning_rate: 4.0688e-04 - gradient_norm: 0.9642 - val_det_loss: 0.1604 - val_cls_loss: 0.1114 - val_box_loss: 9.7953e-04 - val_reg_l2_loss: 0.0924 - val_loss: 0.2528
Epoch 79/90
15/1

In [34]:
%%capture cap --no-stderr
model.evaluate(test_data)

In [35]:
with open(f'evaluation_EfficientDet{model_num}_preLite.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

1/1 [==============================] - 5s 5s/step



{'AP': 0.5211566,
 'AP50': 0.8004745,
 'AP75': 0.568056,
 'APs': 0.4175805,
 'APm': 0.72901905,
 'APl': 0.74534607,
 'ARmax1': 0.022597864,
 'ARmax10': 0.20871887,
 'ARmax100': 0.5911032,
 'ARs': 0.48992044,
 'ARm': 0.7894737,
 'ARl': 0.87333333,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.5211566}

In [36]:
model.export(export_dir='.', tflite_filename=f'model{model_num}.tflite')

2022-08-04 17:00:30.740061: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-08-04 17:00:43.443672: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-08-04 17:00:43.443719: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-08-04 17:00:43.443996: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmphtyzmaut
2022-08-04 17:00:43.635614: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-08-04 17:00:43.635684: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmphtyzmaut
2022-08-04 17:00:44.288650: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-08-04 17:00:47.780364: I tensorflow/cc/saved_

In [37]:
%%capture cap --no-stderr
model.evaluate_tflite(f'model{model_num}.tflite', test_data)

In [38]:
with open(f'evaluation_EfficientDet-Lite{model_num}.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

15/15 [==============================] - 266s 18s/step



{'AP': 0.36480582,
 'AP50': 0.5071582,
 'AP75': 0.42848298,
 'APs': 0.21012376,
 'APm': 0.6759067,
 'APl': 0.75484735,
 'ARmax1': 0.022241993,
 'ARmax10': 0.20729537,
 'ARmax100': 0.39021352,
 'ARs': 0.22387268,
 'ARm': 0.71695906,
 'ARl': 0.86,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.36480582}

## EfficientDet-Lite4

In [39]:
model_num = 4
model_name = f'efficientdet_lite{model_num}'

epochs = 90

## First train the smallest model
spec = model_spec.get(model_name)
spec.config.model_dir = model_name

model = object_detector.create(train_data, 
                               model_spec=spec, 
                               batch_size=8,
                               epochs=epochs,
                               train_whole_model=True, 
                               validation_data=validation_data)

Epoch 1/90
15/15 [==============================] - 83s 2s/step - det_loss: 1.7813 - cls_loss: 1.1125 - box_loss: 0.0134 - reg_l2_loss: 0.1078 - loss: 1.8891 - learning_rate: 0.0090 - gradient_norm: 0.6595 - val_det_loss: 1.5914 - val_cls_loss: 1.0633 - val_box_loss: 0.0106 - val_reg_l2_loss: 0.1078 - val_loss: 1.6992
Epoch 2/90
15/15 [==============================] - 22s 1s/step - det_loss: 1.4542 - cls_loss: 0.9338 - box_loss: 0.0104 - reg_l2_loss: 0.1078 - loss: 1.5620 - learning_rate: 0.0100 - gradient_norm: 1.1529 - val_det_loss: 0.9460 - val_cls_loss: 0.5663 - val_box_loss: 0.0076 - val_reg_l2_loss: 0.1078 - val_loss: 1.0538
Epoch 3/90
15/15 [==============================] - 25s 2s/step - det_loss: 0.8895 - cls_loss: 0.5039 - box_loss: 0.0077 - reg_l2_loss: 0.1078 - loss: 0.9973 - learning_rate: 0.0100 - gradient_norm: 2.2816 - val_det_loss: 0.8261 - val_cls_loss: 0.5873 - val_box_loss: 0.0048 - val_reg_l2_loss: 0.1078 - val_loss: 0.9339
Epoch 4/90
15/15 [======================

15/15 [==============================] - 22s 1s/step - det_loss: 0.2247 - cls_loss: 0.1492 - box_loss: 0.0015 - reg_l2_loss: 0.1077 - loss: 0.3324 - learning_rate: 0.0081 - gradient_norm: 1.1974 - val_det_loss: 0.2010 - val_cls_loss: 0.1396 - val_box_loss: 0.0012 - val_reg_l2_loss: 0.1077 - val_loss: 0.3087
Epoch 27/90
15/15 [==============================] - 22s 2s/step - det_loss: 0.2281 - cls_loss: 0.1472 - box_loss: 0.0016 - reg_l2_loss: 0.1077 - loss: 0.3358 - learning_rate: 0.0080 - gradient_norm: 1.1275 - val_det_loss: 0.1905 - val_cls_loss: 0.1333 - val_box_loss: 0.0011 - val_reg_l2_loss: 0.1077 - val_loss: 0.2983
Epoch 28/90
15/15 [==============================] - 23s 2s/step - det_loss: 0.2341 - cls_loss: 0.1470 - box_loss: 0.0017 - reg_l2_loss: 0.1077 - loss: 0.3418 - learning_rate: 0.0078 - gradient_norm: 1.2205 - val_det_loss: 0.1927 - val_cls_loss: 0.1361 - val_box_loss: 0.0011 - val_reg_l2_loss: 0.1077 - val_loss: 0.3004
Epoch 29/90
15/15 [==============================

15/15 [==============================] - 22s 1s/step - det_loss: 0.1908 - cls_loss: 0.1226 - box_loss: 0.0014 - reg_l2_loss: 0.1076 - loss: 0.2984 - learning_rate: 0.0039 - gradient_norm: 0.8829 - val_det_loss: 0.1662 - val_cls_loss: 0.1181 - val_box_loss: 9.6222e-04 - val_reg_l2_loss: 0.1076 - val_loss: 0.2738
Epoch 52/90
15/15 [==============================] - 22s 1s/step - det_loss: 0.1843 - cls_loss: 0.1223 - box_loss: 0.0012 - reg_l2_loss: 0.1076 - loss: 0.2919 - learning_rate: 0.0038 - gradient_norm: 0.8570 - val_det_loss: 0.1689 - val_cls_loss: 0.1196 - val_box_loss: 9.8617e-04 - val_reg_l2_loss: 0.1076 - val_loss: 0.2765
Epoch 53/90
15/15 [==============================] - 22s 1s/step - det_loss: 0.1878 - cls_loss: 0.1203 - box_loss: 0.0013 - reg_l2_loss: 0.1076 - loss: 0.2954 - learning_rate: 0.0036 - gradient_norm: 0.9353 - val_det_loss: 0.1679 - val_cls_loss: 0.1184 - val_box_loss: 9.9103e-04 - val_reg_l2_loss: 0.1076 - val_loss: 0.2755
Epoch 54/90
15/15 [==================

15/15 [==============================] - 23s 2s/step - det_loss: 0.1862 - cls_loss: 0.1197 - box_loss: 0.0013 - reg_l2_loss: 0.1075 - loss: 0.2938 - learning_rate: 5.5764e-04 - gradient_norm: 0.9128 - val_det_loss: 0.1655 - val_cls_loss: 0.1171 - val_box_loss: 9.6892e-04 - val_reg_l2_loss: 0.1075 - val_loss: 0.2731
Epoch 77/90
15/15 [==============================] - 23s 2s/step - det_loss: 0.1921 - cls_loss: 0.1229 - box_loss: 0.0014 - reg_l2_loss: 0.1075 - loss: 0.2996 - learning_rate: 4.7944e-04 - gradient_norm: 0.9146 - val_det_loss: 0.1650 - val_cls_loss: 0.1171 - val_box_loss: 9.5946e-04 - val_reg_l2_loss: 0.1075 - val_loss: 0.2726
Epoch 78/90
15/15 [==============================] - 22s 2s/step - det_loss: 0.1736 - cls_loss: 0.1172 - box_loss: 0.0011 - reg_l2_loss: 0.1075 - loss: 0.2811 - learning_rate: 4.0688e-04 - gradient_norm: 0.9050 - val_det_loss: 0.1646 - val_cls_loss: 0.1169 - val_box_loss: 9.5444e-04 - val_reg_l2_loss: 0.1075 - val_loss: 0.2722
Epoch 79/90
15/15 [======

In [40]:
%%capture cap --no-stderr
model.evaluate(test_data)

In [41]:
with open(f'evaluation_EfficientDet{model_num}_preLite.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

1/1 [==============================] - 7s 7s/step



{'AP': 0.57953405,
 'AP50': 0.85734445,
 'AP75': 0.6351488,
 'APs': 0.48923826,
 'APm': 0.7688668,
 'APl': 0.7227164,
 'ARmax1': 0.022064056,
 'ARmax10': 0.21316727,
 'ARmax100': 0.65355873,
 'ARs': 0.5618037,
 'ARm': 0.8350877,
 'ARl': 0.9,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.57953405}

In [42]:
model.export(export_dir='.', tflite_filename=f'model{model_num}.tflite')

2022-08-04 17:45:17.712138: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-08-04 17:45:36.152348: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-08-04 17:45:36.152416: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-08-04 17:45:36.152798: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpc6359r0w
2022-08-04 17:45:36.387858: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-08-04 17:45:36.387918: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpc6359r0w
2022-08-04 17:45:37.169449: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-08-04 17:45:41.816989: I tensorflow/cc/saved_

In [43]:
%%capture cap --no-stderr
model.evaluate_tflite(f'model{model_num}.tflite', test_data)

In [44]:
with open(f'evaluation_EfficientDet-Lite{model_num}.txt', 'w') as f:
    f.write("\n".join(str(cap.outputs[0].data).split('\\n')))
    
cap()

15/15 [==============================] - 718s 48s/step



{'AP': 0.39057598,
 'AP50': 0.5221556,
 'AP75': 0.45490608,
 'APs': 0.24765071,
 'APm': 0.6898592,
 'APl': 0.6401355,
 'ARmax1': 0.021352313,
 'ARmax10': 0.2119217,
 'ARmax100': 0.4163701,
 'ARs': 0.2607427,
 'ARm': 0.7245614,
 'ARl': 0.7866667,
 'AP_/label': -1.0,
 'AP_/climbing_hold': 0.39057598}